# **🎯 GPT 기반 정량평가 : : QG-QA**
## **📍문제 정의**
- 뉴스 기사 각각에 대해 LLM을 적용하여 기사에 대한 블로그 글을 만든 상황에서,
- 블로그 글이 뉴스 기사의 내용을 잘 담고 있는지 QG-QA를 통해 평가하고자 한다.
- 사용 모델: **GPT-4**
1. **뉴스 기사와 그에 대응하는 블로그 글에 대해 동일한 질문을 생성**
  - OUTPUT 이 질문 리스트가 되도록 PROMPT 구성하기
  - 질문은 다음과 같은 것들이어야 한다:
    - "지연된 노선은 3, 4, 5호선이 맞나요?"
    - "시간표 변경은 5월 27일부터 적용되나요?"
    - "언제 지하철 파업이 시작되었나요?"
2. **뉴스 기사, 블로그 글에 질문 각각 ASK**
3. **그에 대한 답변이 서로 일치하는지 확인한다.**
  - 두 답변을 서로 비교하여 일치할 경우 True(1), 아닐 경우 False(0) 반환


        * (1) 입력 - 뉴스 제목, 뉴스 본문, 뉴스 날짜, 블로그 텍스트 → news_sample, blog_sample
        * (2) RPOMPT1 - 뉴스기사를 기반으로 질문을 만들어줘- 질문 도출
        * (3) PROMPT2 - 뉴스 기사에 대해 질문의 답변 도출
        * (4) PROMPT3 - 블로그 글에 대해 질문의 답변 도출
        * (5)일치여부판단 - (3), (4)의 결과로 나온 답변을 비교

# **📍1. 환경세팅**

In [7]:
# 구글 드라이브 연결
from google.colab import drive
drive.mount('/content/drive')

# 데이터 불러오기
import pandas as pd
news_data = pd.read_csv("/content/drive/My Drive/DSCapstone/Newsdata_URL_TEXT.csv", index_col=0)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
!pip install langchain

In [9]:
!pip install langchain_openai

In [10]:
import getpass
import os
# 환경변수에 OpenAI API 키 저장 (사용자 입력으로 안전하게)
os.environ["OPENAI_API_KEY"] = getpass.getpass()

··········


# **📍2.입력 데이터 준비**

* **2-1. 뉴스 제목, 날짜, 본문 데이터**

In [11]:
# 우선은 1개의 뉴스-블로그 쌍에 대해서만 적용
news_title = news_data.iloc[0]['TITLE'] # 제목
news_date = news_data.iloc[0]['DATE'] # 날짜
news_article = news_data.iloc[0]['ARTICLE'] # 본문
news_sample = f"""<뉴스 제목>: {news_title},
<뉴스 생성일자>: {news_date},
<뉴스 본문>: {news_article}""" # 뉴스 샘플 = 뉴스 제목+날짜+본문

In [12]:
# 하나의 뉴스 기사 샘플 데이터
news_sample

'<뉴스 제목>: 서울 1~8호선 지하철, 오늘 오전 9시부터 ‘경고 파업’, \n<뉴스 생성일자>: 2023-11-08 22:20:00,\n<뉴스 본문>:  서울지하철 1~8호선을 운영하는 서울교통공사(공사) 노동조합이 9일 아침 9시부터 10일 오후 6시까지 이틀 동안 경고 파업에 돌입한다. 회사가 제시한 인력감축·외주화 문제를 두고 노사가 8일 밤까지 벌인 막판 교섭에서도 타협점을 찾지 못한 결과다. 공사는 노조가 파업에 돌입해도 필수 유지업무 인력과 대체 인력 등을 활용해 80% 이상의 운행률을 유지할 방침이다.\n\n공사는 8일 밤 “노사 간 협상이 최종 결렬됐다”며 “양대노총 소속 서울교통공사 노조들로 구성된 연합교섭단은 9일 오전 9시부터 10일 저녁 6시까지 (서울시와 사쪽의 입장 변화를 촉구하는 의미) 경고 파업에 돌입한다”고 밝혔다. 이날 오후 3시부터 교섭단과 공사는 막판 교섭을 시작했으나 타협점을 찾지 못했다.\n\n핵심 쟁점은 ‘인력 감축’이었다. 공사는 최근 2026년까지 2212명의 인력을 줄이겠다는 계획을 노조에 제시했다. 이는 현재 공사 전체 정원의 약 13.5% 수준이다. 공사는 연 1조원 안팎의 대규모 적자를 내고 있어 인력 감축이 불가피하다는 입장이다. 반면 연합교섭단은 무리한 인력 감축이 시민의 안전 문제와 직결될 수 있다며 감축안 철회를 요구했다.\n\n노사가 맺은 필수 유지업무 실무 협정에 따라 파업 기간 1~4호선 65.7%, 5~8호선 79.8%의 운행률은 유지돼야 한다. 서울교통공사쪽은 “9일부터 경고파업에 돌입하더라도 파업 대비 정상운행 대책본부를 운영해 열차 정상 운행과 시설물 안전 확보 등 대책을 마련해 시민의 불편을 최소화할 계획”이라며 ““(파업 돌입 시) 필수유지업무 인력과 대체 인력 등을 확보해 오전 7시∼9시 출근길에는 100%, 나머지 시간에는 80% 정도 수준의 운행률로 시민 불편을 최소화할 것”이라고 말했다.'

* **2-2. 뉴스 블로그 글  데이터**

In [13]:
blog_sample = """ 서울 지하철 파업 경보: 1~8호선 운행 차질 예고!
안녕하세요, 여러분의 출퇴근 메신저 지하철 온다의 '오.지.통 [오늘의 지하철 소식통]' 인사 드립니다!
오늘 아침, 서울의 출근길이 조금 달라졌습니다. 서울지하철 1~8호선을 운영하는 서울교통공사의 노동조합이 2023년 11월 9일 아침 9시부터 10일 오후 6시까지 이틀 간 경고 파업에 돌입했습니다. 이는 노사 간의 인력감축 및 외주화 문제를 두고 벌어진 교섭이 결렬된 결과입니다.

사건 일시: 2023년 11월 9일 오전 9시부터 11월 10일 오후 6시까지
사건 지하철 노선: 서울 지하철 1~8호선
사건 발생 원인: 인력 감축 및 외주화 문제에 대한 노사 간 최종 교섭 결렬
공사는 파업 동안 필수 유지 업무 인력과 대체 인력을 활용하여 14호선은 65.7%, 58호선은 79.8%의 운행률을 유지할 계획입니다. 특히 출근 시간대에는 100%에 가까운 운행률을 목표로 하고 있으나, 여전히 출퇴근길 혼잡이 예상됩니다.

지하철 운행 차질에 대한 간단 요약
2023년 11월 9일과 10일에 걸쳐 서울 지하철 1~8호선에서 경고 파업이 실시되며, 이는 노사 간 인력 감축 및 외주화 문제에 대한 논의 실패로 인한 것입니다. 파업 기간 동안 일부 노선의 운행률이 저하되어 출퇴근 시간대의 혼잡이 가중될 것으로 보입니다. 서울교통공사는 필수 유지 업무 인력을 활용하여 운행률을 최대한 유지하려 하고 있습니다.

출근길이 힘들어지셨죠? 파업으로 인한 지하철 운행 차질로 여러분의 하루가 더욱 버거워졌을 텐데요, 저희 '지하철 온다' 앱을 활용하시면 더욱 효율적으로 대처하실 수 있습니다!
오지통이 실시간으로 다양한 지하철 정보를 업데이트 할 예정이니, 자주 방문해 주세요. 지하철과 함께라면 어디든지 즐거운 여정이 될 거예요. 🚆💨

🔽 지하철 온다 소개 보러가기

http://www.letskorail.com

이번 파업은 단순한 교통 불편 문제를 넘어, 우리 모두가 함께 고민하고 해결해 나가야 할 사회적 이슈입니다. 서로를 배려하며,공동의 이익을 위해 논의하는 자세가 중요합니다. 서울교통공사와 노조 양측이 상호 이해와 협력을 통해 더 나은 해결책을 모색하길 바라며, 이 과정에서 시민들의 안전과 편의가 최우선으로 고려되어야 할 것입니다. 모두가 조금씩 양보하고 소통하여, 이번 경고 파업이 조속히 마무리되어 일상이 원활하게 회복되길 기원합니다. 이러한 상황에서 '지하철 온다' 앱이 실시간으로 변경되는 교통 상황을 알려드리고, 여러분이 좀 더 편리하게 대처할 수 있도록 도움을 제공하겠습니다."""

# **📍3.(메모리 미사용)Question Generation & Question Answering**

In [19]:
# GPT-4 챗봇 객체 생성
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4-turbo")

# 프롬프트 템플릿, 파서를 각각 생성한 뒤 챗봇과 체인 결합
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
prompt = ChatPromptTemplate.from_messages([
    ("system", ""), # 페르소나 부여
    ("user", "{input}")
])
output_parser = StrOutputParser()
chain = prompt | llm | output_parser

# PROMPT1 입력 - 뉴스 기반으로 질문 만들어줘
prompt_1 = ChatPromptTemplate.from_messages([
    ("system", """너는 뉴스 기사를 읽고, 뉴스의 핵심 내용들에 대해서 오지선다 질문을 던지는 것을 잘 해."""), # 페르소나 부여
    ("user", """제시된 서울 지하철 뉴스를 읽고, 뉴스의 주요 내용(사건 일시, 사건 지하철 호선, 사건 발생 원인)을 확인하기 위한 3가지 질문을 만들어줘. 이 때, 질문은 1번부터 5번까지의 선지 중에서 하나를 고르는 5지선다형 문제여야만해.
    예시= '해당 지하철 사건이 일어난 발생지는 어디 인가요? (1)강남역 (2)성수역 (3)수유역 (4)안국역 (5)홍대입구역)
    뉴스= {input}
    """),
    ("system", """템플릿:
    질문1: {{question_1}},
    질문2: {{question_2}},
    질문3: {{question_3}}""")
])
chain_1 = prompt_1 | llm | output_parser
question_list = chain_1.invoke({"input": news_sample}) # 활용




In [27]:

# PROMPT2 입력 - 뉴스 기사에 대해 질문의 답변 도출
prompt_2 = ChatPromptTemplate.from_messages([
    ("system", """너는 뉴스 기사를 읽고, 읽은 내용을 바탕으로 질문에 대해 정확히 답변해"""), # 페르소나 부여
    ("user", """제시된 서울 지하철 뉴스와 해당 뉴스에 대한 3가지 질문에 대해 답변해줘.
    예시= "1번, 5번, 4번",
    뉴스= {input}
    질문= {question}
    """),
    ("system", """템플릿:
    답변1: {{answer_1}}번,
    답변2: {{answer_2}}번,
    답변3: {{answer_3}}번""")
])
chain_2 = prompt_2 | llm | output_parser
answer_list_news = chain_2.invoke({"input": news_sample, "question": question_list}) # 활용


In [28]:
print(question_list)
print(answer_list)

질문1: 이번 서울 지하철 경고 파업이 시작되는 시간은 언제입니까?
(1) 오전 7시 (2) 오전 8시 (3) 오전 9시 (4) 오전 10시 (5) 오전 11시

질문2: 이번 파업이 적용되는 서울 지하철의 호선은 어디까지 입니까?
(1) 1~2호선 (2) 1~4호선 (3) 1~6호선 (4) 1~8호선 (5) 1~10호선

질문3: 이번 파업의 주요 원인은 무엇 때문입니까?
(1) 임금 삭감 (2) 근무 조건 개선 요구 (3) 인력 감축 (4) 운영 시간 변경 (5) 열차 운행 감소
답변1: 3번,
답변2: 4번,
답변3: 3번


In [30]:
# PROMPT3 입력 - 블로그글에 대해 질문의 답변 도출
prompt_3 = ChatPromptTemplate.from_messages([
    ("system", """너는 블로그 글을 읽고, 읽은 내용을 바탕으로 질문에 대해 정확히 답변해"""), # 페르소나 부여
    ("user", """제시된 서울 지하철 뉴스와 해당 뉴스에 대한 3가지 질문에 대해 답변해줘.
    예시= "1번, 5번, 4번",
    블로그 글= {input}
    질문= {question}
    """),
    ("system", """템플릿:
    답변1: {{answer_1}}번,
    답변2: {{answer_2}}번,
    답변3: {{answer_3}}번""")
])
chain_3 = prompt_3 | llm | output_parser
answer_list_blog = chain_3.invoke({"input": blog_sample, "question": question_list}) # 활용

In [32]:
print(question_list)
print(answer_list_blog)

질문1: 이번 서울 지하철 경고 파업이 시작되는 시간은 언제입니까?
(1) 오전 7시 (2) 오전 8시 (3) 오전 9시 (4) 오전 10시 (5) 오전 11시

질문2: 이번 파업이 적용되는 서울 지하철의 호선은 어디까지 입니까?
(1) 1~2호선 (2) 1~4호선 (3) 1~6호선 (4) 1~8호선 (5) 1~10호선

질문3: 이번 파업의 주요 원인은 무엇 때문입니까?
(1) 임금 삭감 (2) 근무 조건 개선 요구 (3) 인력 감축 (4) 운영 시간 변경 (5) 열차 운행 감소
답변1: 3번,
답변2: 4번,
답변3: 3번


In [33]:
print(answer_list_news == answer_list_blog)

True


# **📍4.함수화**
- 입력: 뉴스의 제목, 날짜, 본문, 그리고 뉴스를 기반으로 만든 블로그 글
- 출력: 3가지 질문에 대해 뉴스, 블로그의 답변이 모두 일치하면 True, 아니면 False 반환


In [34]:
def temp_function(title, date, article, blog):

  # news = 제목 + 날짜 + 본문
  news = f"""<뉴스 제목>: {news_title},
  <뉴스 생성일자>: {news_date},
  <뉴스 본문>: {news_article}"""

  # news 기반으로 질문 3개 생성
  question_list = chain_1.invoke({"input": news})

  # 질문 3개에 대해 news 기반으로 답변
  answer_list_news = chain_2.invoke({"input": news, "question": question_list})

  # 질문 3개에 대해 blog 기반으로 답변
  answer_list_blog = chain_3.invoke({"input": blog, "question": question_list})

  # 일치 여부 판단(True/False)
  return answer_list_news == answer_list_blog

In [35]:
# 작동 확인(19초 소요)
temp_function(news_title, news_date, news_article, blog_sample)

True

# **📍5.1~4 내용 단일 셀로 정리**

In [41]:
# GPT-4 챗봇 객체 생성
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4-turbo")

# 프롬프트 템플릿, 파서를 각각 생성한 뒤 챗봇과 체인 결합
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()

# PROMPT1 - 뉴스 기반으로 질문 생성 - chain_1으로 생성
prompt_1 = ChatPromptTemplate.from_messages([
    ("system", """너는 뉴스 기사를 읽고, 뉴스의 핵심 내용들에 대해서 오지선다 질문을 던지는 것을 잘 해."""), # 페르소나 부여
    ("user", """제시된 서울 지하철 뉴스를 읽고, 뉴스의 주요 내용(사건 일시, 사건 지하철 호선, 사건 발생 원인)을 확인하기 위한 3가지 질문을 만들어줘. 이 때, 질문은 1번부터 5번까지의 선지 중에서 하나를 고르는 5지선다형 문제여야만해.
    예시= '해당 지하철 사건이 일어난 발생지는 어디 인가요? (1)강남역 (2)성수역 (3)수유역 (4)안국역 (5)홍대입구역)
    뉴스= {input}
    """),
    ("system", """템플릿:
    질문1: {{question_1}},
    질문2: {{question_2}},
    질문3: {{question_3}}""")
])
chain_1 = prompt_1 | llm | output_parser

# PROMPT2 입력 - 뉴스 기사에 대해 질문의 답변 도출 - chain_2으로 생성
prompt_2 = ChatPromptTemplate.from_messages([
    ("system", """너는 뉴스 기사를 읽고, 읽은 내용을 바탕으로 질문에 대해 정확히 답변해"""), # 페르소나 부여
    ("user", """제시된 서울 지하철 뉴스와 해당 뉴스에 대한 3가지 질문에 대해 답변해줘.
    예시= "1번, 5번, 4번",
    뉴스= {input}
    질문= {question}
    """),
    ("system", """템플릿:
    답변1: {{answer_1}}번,
    답변2: {{answer_2}}번,
    답변3: {{answer_3}}번""")
])
chain_2 = prompt_2 | llm | output_parser

# PROMPT3 입력 - 블로그글에 대해 질문의 답변 도출 - chain_3으로 생성
prompt_3 = ChatPromptTemplate.from_messages([
    ("system", """너는 블로그 글을 읽고, 읽은 내용을 바탕으로 질문에 대해 정확히 답변해"""), # 페르소나 부여
    ("user", """제시된 서울 지하철 뉴스와 해당 뉴스에 대한 3가지 질문에 대해 답변해줘.
    예시= "1번, 5번, 4번",
    블로그 글= {input}
    질문= {question}
    """),
    ("system", """템플릿:
    답변1: {{answer_1}}번,
    답변2: {{answer_2}}번,
    답변3: {{answer_3}}번""")
])
chain_3 = prompt_3 | llm | output_parser



# 함수(제목, 날짜, 본문이 따로 추출된 경우)
def quan_eval_1(title, date, article, blog):
  # news = 제목 + 날짜 + 본문
  news = f"""<뉴스 제목>: {news_title},
  <뉴스 생성일자>: {news_date},
  <뉴스 본문>: {news_article}"""

  # news 기반으로 질문 3개 생성
  question_list = chain_1.invoke({"input": news})

  # 질문 3개에 대해 news 기반으로 답변
  answer_list_news = chain_2.invoke({"input": news, "question": question_list})

  # 질문 3개에 대해 blog 기반으로 답변
  answer_list_blog = chain_3.invoke({"input": blog, "question": question_list})

  # 일치 여부 판단(True/False)
  return answer_list_news == answer_list_blog



# 함수(제목, 날짜, 본문이 따로 추출된 경우)
def quan_eval_2(news, blog):

  # news 기반으로 질문 3개 생성
  question_list = chain_1.invoke({"input": news})

  # 질문 3개에 대해 news 기반으로 답변
  answer_list_news = chain_2.invoke({"input": news, "question": question_list})

  # 질문 3개에 대해 blog 기반으로 답변
  answer_list_blog = chain_3.invoke({"input": blog, "question": question_list})

  # 일치 여부 판단(True/False)
  return answer_list_news == answer_list_blog

### **✦ 논의사항**
- 하나의 기사-블로그 쌍에 대해 **질문 몇 개씩** 만들건지?
    - 현재 **3**개로 설정해 둔 상태

- **점수 부여**는 어떤 식으로 진행할 것인지?
        * 세개 중에서 맞는 것도, 틀린 것도 생길 것.
            * 후보1) 하나라도 틀리면 False 로 처리 > 현재 이렇게 설정해 둔 상태
            * 후보2) 1개까지는/2개까지.... 는 틀려도 True로 처리
            * 후보3) 틀린 개수에 비례하게 총점에서 감점시키기


# **📍6. 다른 뉴스-블로그 쌍에 적용**

- 가장 최근에 업데이트 된 프롬프트로 만든 블로그글로 테스트
- 사용한 파일 명세
  - 파업글: strikeresult.csv
  - 연착글: delayresult.csv
  - 시간표글: timeresult.csv

## **6-1. 데이터 불러오기**

In [36]:
strike_data = pd.read_csv("/content/drive/MyDrive/DSCapstone/strikresult.csv", index_col=0) # 파업
delay_data = pd.read_csv("/content/drive/MyDrive/DSCapstone/delayresult.csv", index_col=0) # 지연
time_data = pd.read_csv("/content/drive/MyDrive/DSCapstone/timeresult.csv", index_col=0) # 시간표

In [37]:
strike_data.head(1)

text  \
0  \n“파업소식에 1시간 일찍 출근”…기차 취소돼 급히 버스 예매도\n중앙일보\n입력...   

                                              output Consistency Coherence  \
0  **교통 대란 주의보! 전국 철도 파업, 대체 교통수단은?**\n\n안녕하세요, 여...  {score: 5}         4   

   Friendliness  Fluency  Humanlikeness Readability  
0             3        5              4           4

In [38]:
delay_data.head(1)

text  \
0  \n\n"월요일부터 지각" 지하철 1호선 지연 운행에 시민 불편…"노조 준법투쟁"\...   

                                              output  Consistency  \
0  **출근길 지각 불가피: 지하철 1호선 연착 사태로 시민 분통!**\n안녕하세요, ...            5   

                                           Coherence  Friendliness  Fluency  \
0  Coherence: 4\n\nThe blog content maintains a g...             3        5   

   Humanlikeness Readability  
0              4           3

In [40]:
time_data.head(1)

text  \
0  \n제목\t1호선 열차운영계획 추가 조정 알림(경인급행열차)\n작성일\t2024-0...   

                                              output Consistency  \
0  **경인급행, 새로운 시간표로 더 빠르게! 1호선 열차운행계획 추가 조정 소식**\...           5   

      Coherence  Friendliness  Fluency  Humanlikeness  \
0  Coherence: 4             4        5              4   

                                         Readability  
0  **Evaluation:**\n\n1. **Sentence length:** The...

### test #1. 파업글에 적용

In [42]:
quan_eval_2(strike_data.head(1)['text'],strike_data.head(1)['output'])

False

### test #2 지연글에 적용

In [43]:
quan_eval_2(delay_data.head(1)['text'],delay_data.head(1)['output'])

False

### test #3 시간표글에 적용

In [44]:
quan_eval_2(time_data.head(1)['text'],time_data.head(1)['output'])

False

## **6-2. 결론: 음,,?고쳐보자 뭐가 문제지**



In [47]:
# 결과까지 다 볼 수 있게 함수 새로 정의
def quan_eval_3(news, blog):

  # news 기반으로 질문 3개 생성
  question_list = chain_1.invoke({"input": news})

  # 질문 3개에 대해 news 기반으로 답변
  answer_list_news = chain_2.invoke({"input": news, "question": question_list})

  # 질문 3개에 대해 blog 기반으로 답변
  answer_list_blog = chain_3.invoke({"input": blog, "question": question_list})

  # 일치 여부 판단(True/False)
  return answer_list_news == answer_list_blog, '\n', question_list, '\n', answer_list_news,'\n', answer_list_blog

In [48]:
quan_eval_3(time_data.head(1)['text'],time_data.head(1)['output'])

(False,
 '\n',
 '질문1: 해당 지하철 사건이 일어난 호선은 어디인가요? (1) 1호선 (2) 2호선 (3) 3호선 (4) 4호선 (5) 5호선\n질문2: 해당 지하철 사건의 주요 원인은 무엇인가요? (1) 자연재해 (2) 기술적 결함 (3) 운행 계획 변경 (4) 승객 문제 (5) 직원 파업\n질문3: 해당 사건이 발생한 시기는 언제인가요? (1) 2022년 (2) 2023년 (3) 2024년 (4) 2025년 (5) 2026년',
 '\n',
 '답변1: 1번,\n답변2: 3번,\n답변3: 3번',
 '\n',
 '답변1: 1번,\n답변2: 3번,\n답변3: 정보 제공 없음')

In [51]:
time_data.head(1)

text  \
0  \n제목\t1호선 열차운영계획 추가 조정 알림(경인급행열차)\n작성일\t2024-0...   

                                              output Consistency  \
0  **경인급행, 새로운 시간표로 더 빠르게! 1호선 열차운행계획 추가 조정 소식**\...           5   

      Coherence  Friendliness  Fluency  Humanlikeness  \
0  Coherence: 4             4        5              4   

                                         Readability  
0  **Evaluation:**\n\n1. **Sentence length:** The...

* 블로그에도 분명 24년 5월 1일이라고 적혀있는데 GPT 가 실수한듯 함[링크 텍스트](https://)

In [52]:
quan_eval_3(delay_data.head(1)['text'],delay_data.head(1)['output'])

(False,
 '\n',
 '질문1: 해당 지하철 사건이 일어난 날짜는 언제입니까?\n(1) 월요일 (2) 화요일 (3) 수요일 (4) 목요일 (5) 금요일\n\n질문2: 사건이 발생한 지하철 호선은 무엇입니까?\n(1) 1호선 (2) 2호선 (3) 3호선 (4) 4호선 (5) 5호선\n\n질문3: 이번 사건의 주된 원인은 무엇입니까?\n(1) 기계 고장 (2) 태풍 (3) 노조 준법투쟁 (4) 열차 사고 (5) 전력 문제',
 '\n',
 '답변1: 1번,\n답변2: 1번,\n답변3: 3번',
 '\n',
 '답변1: 월요일부터 금요일까지의 날짜가 명시되지 않았으므로 정확한 요일을 알 수 없습니다.\n\n답변2: 1번,\n\n답변3: 해당 블로그 글에서는 지하철 연착의 구체적인 원인을 언급하지 않았습니다.')

In [53]:
delay_data.head(1)

text  \
0  \n\n"월요일부터 지각" 지하철 1호선 지연 운행에 시민 불편…"노조 준법투쟁"\...   

                                              output  Consistency  \
0  **출근길 지각 불가피: 지하철 1호선 연착 사태로 시민 분통!**\n안녕하세요, ...            5   

                                           Coherence  Friendliness  Fluency  \
0  Coherence: 4\n\nThe blog content maintains a g...             3        5   

   Humanlikeness Readability  
0              4           3

In [54]:
quan_eval_3(strike_data.head(1)['text'],strike_data.head(1)['output'])

(False,
 '\n',
 '질문1: 이번 서울 지하철 사건은 어떤 원인으로 발생했나요?\n(1) 기계 고장 (2) 승객 사고 (3) 날씨 영향 (4) 파업 (5) 전력 공급 문제\n\n질문2: 해당 지하철 사건이 발생한 시간은 언제인가요?\n(1) 아침 7시 (2) 점심 12시 (3) 오후 3시 (4) 저녁 6시 (5) 새벽 1시\n\n질문3: 해당 지하철 사건이 발생한 서울의 지하철 호선은 어디인가요?\n(1) 1호선 (2) 2호선 (3) 3호선 (4) 4호선 (5) 5호선',
 '\n',
 '답변1: 4번,\n답변2: 5번,\n답변3: 정보가 제공되지 않았습니다.',
 '\n',
 '답변1: 4번,\n답변2: 정보가 부족하여 정확한 답변을 할 수 없습니다,\n답변3: 정보가 부족하여 정확한 답변을 할 수 없습니다')

In [55]:
strike_data.head(1)

text  \
0  \n“파업소식에 1시간 일찍 출근”…기차 취소돼 급히 버스 예매도\n중앙일보\n입력...   

                                              output Consistency Coherence  \
0  **교통 대란 주의보! 전국 철도 파업, 대체 교통수단은?**\n\n안녕하세요, 여...  {score: 5}         4   

   Friendliness  Fluency  Humanlikeness Readability  
0             3        5              4           4

### **발견한 문제:**
* 틀린 정보가 아니라 '누락된' 정보여도 False 로 출력됨
* '모름' 또는 '알 수 없음'이라는 선지를 고정 선지로 취해야함
* 날짜정보의 선지가 너무 디테일할 경우(년원일 말고 요일, 시간 등이 나와있을 경우) '그런 정보 없음'이라고 모르쇠로 나오는 경우 많음 -> 역시 False 로 출력됨
* **음..아니면 질문이나 선지 수, 채점 방식을 바꿔야하나?**

## **6-3. 수정본 단일 셀**
1. **선지에 (5) '알 수 없음' 무조건 포함**
2. **일자정보 질문의 선지 형식 2018-01-01 형식으로 통일**

In [56]:
# PROMPT1 - 뉴스 기반으로 질문 생성 - chain_1으로 생성
prompt_1 = ChatPromptTemplate.from_messages([
    ("system", """너는 뉴스 기사를 읽고, 뉴스의 핵심 내용들에 대해서 오지선다 질문을 던지는 것을 잘 해."""), # 페르소나 부여
    ("user", """제시된 서울 지하철 뉴스를 읽고, 뉴스의 주요 내용(사건 일시, 사건 지하철 호선, 사건 발생 원인)을 확인하기 위한 3가지 질문을 만들어줘. 이 때, 질문은 1번부터 5번까지의 선지 중에서 하나를 고르는 5지선다형 문제여야만해.
    예시= '해당 지하철 사건이 일어난 발생지는 어디인가요? (1)강남역 (2)성수역 (3)수유역 (4)안국역 (5)알 수 없음). 단, 선지 중 하나는 반드시 '알 수 없음' 이어야 하고, '사건 일시' 질문의 선지는 2018-03-08 형식을 갖춰야해.
    뉴스= {input}
    """),
    ("system", """템플릿:
    질문1: {{question_1}},
    질문2: {{question_2}},
    질문3: {{question_3}}""")
])
chain_1 = prompt_1 | llm | output_parser

# PROMPT2 입력 - 뉴스 기사에 대해 질문의 답변 도출 - chain_2으로 생성
prompt_2 = ChatPromptTemplate.from_messages([
    ("system", """너는 뉴스 기사를 읽고, 읽은 내용을 바탕으로 질문에 대해 정확히 답변해"""), # 페르소나 부여
    ("user", """제시된 서울 지하철 뉴스와 해당 뉴스에 대한 3가지 질문에 대해 답변해줘.
    예시= "1번, 5번, 4번",
    뉴스= {input}
    질문= {question}
    """),
    ("system", """템플릿:
    답변1: {{answer_1}}번,
    답변2: {{answer_2}}번,
    답변3: {{answer_3}}번""")
])
chain_2 = prompt_2 | llm | output_parser

# PROMPT3 입력 - 블로그글에 대해 질문의 답변 도출 - chain_3으로 생성
prompt_3 = ChatPromptTemplate.from_messages([
    ("system", """너는 블로그 글을 읽고, 읽은 내용을 바탕으로 질문에 대해 정확히 답변해"""), # 페르소나 부여
    ("user", """제시된 서울 지하철 뉴스와 해당 뉴스에 대한 3가지 질문에 대해 답변해줘.
    예시= "1번, 5번, 4번",
    블로그 글= {input}
    질문= {question}
    """),
    ("system", """템플릿:
    답변1: {{answer_1}}번,
    답변2: {{answer_2}}번,
    답변3: {{answer_3}}번""")
])
chain_3 = prompt_3 | llm | output_parser



# 함수(제목, 날짜, 본문이 따로 추출된 경우)
def quan_eval_1(title, date, article, blog):
  # news = 제목 + 날짜 + 본문
  news = f"""<뉴스 제목>: {news_title},
  <뉴스 생성일자>: {news_date},
  <뉴스 본문>: {news_article}"""

  # news 기반으로 질문 3개 생성
  question_list = chain_1.invoke({"input": news})

  # 질문 3개에 대해 news 기반으로 답변
  answer_list_news = chain_2.invoke({"input": news, "question": question_list})

  # 질문 3개에 대해 blog 기반으로 답변
  answer_list_blog = chain_3.invoke({"input": blog, "question": question_list})

  # 일치 여부 판단(True/False)
  return answer_list_news == answer_list_blog



# 함수(제목, 날짜, 본문이 따로 추출된 경우)
def quan_eval_2(news, blog):

  # news 기반으로 질문 3개 생성
  question_list = chain_1.invoke({"input": news})

  # 질문 3개에 대해 news 기반으로 답변
  answer_list_news = chain_2.invoke({"input": news, "question": question_list})

  # 질문 3개에 대해 blog 기반으로 답변
  answer_list_blog = chain_3.invoke({"input": blog, "question": question_list})

  # 일치 여부 판단(True/False)
  return answer_list_news == answer_list_blog

In [57]:
# 결과까지 다 볼 수 있게 함수 새로 정의
def quan_eval_3(news, blog):

  # news 기반으로 질문 3개 생성
  question_list = chain_1.invoke({"input": news})

  # 질문 3개에 대해 news 기반으로 답변
  answer_list_news = chain_2.invoke({"input": news, "question": question_list})

  # 질문 3개에 대해 blog 기반으로 답변
  answer_list_blog = chain_3.invoke({"input": blog, "question": question_list})

  # 일치 여부 판단(True/False)
  return answer_list_news == answer_list_blog, '\n', question_list, '\n', answer_list_news,'\n', answer_list_blog

## test #1. 파업글에 적용

In [59]:
quan_eval_3(strike_data.head(1)['text'],strike_data.head(1)['output'])

(True,
 '\n',
 '질문1: 해당 지하철 사건이 일어난 날짜는 언제인가요?\n(1) 2023-01-15\n(2) 2023-02-20\n(3) 2023-03-30\n(4) 2023-04-05\n(5) 알 수 없음\n\n질문2: 해당 사건이 발생한 지하철 호선은 무엇인가요?\n(1) 1호선\n(2) 2호선\n(3) 3호선\n(4) 4호선\n(5) 알 수 없음\n\n질문3: 이 지하철 사건의 발생 원인은 무엇인가요?\n(1) 기계 고장\n(2) 인력 부족\n(3) 파업\n(4) 날씨 영향\n(5) 알 수 없음',
 '\n',
 '답변1: 5번,\n답변2: 5번,\n답변3: 3번',
 '\n',
 '답변1: 5번,\n답변2: 5번,\n답변3: 3번')

In [63]:
quan_eval_3(strike_data.tail(1)['text'],strike_data.tail(1)['output'])

(True,
 '\n',
 '질문1: 전장연이 평일 1년 내내 시위를 계획한 지하철 호선은 어디인가요? \n(1) 1호선 (2) 2호선 (3) 3호선 (4) 4호선 (5) 알 수 없음\n\n질문2: 오세훈 시장의 "무관용 강력 대응"이 언급된 사건의 발생 원인은 무엇인가요?\n(1) 열차 지연 (2) 열차 고장 (3) 시위 (4) 승객 간의 분쟁 (5) 알 수 없음\n\n질문3: 전장연의 시위 계획 발표가 언급된 뉴스 기사의 날짜는 언제인가요?\n(1) 2023-01-15 (2) 2023-02-01 (3) 2023-03-01 (4) 2023-04-01 (5) 알 수 없음',
 '\n',
 '답변1: 4번,\n답변2: 3번,\n답변3: 5번',
 '\n',
 '답변1: 4번,\n답변2: 3번,\n답변3: 5번')

### test #2. 지연글에 적용📌(False 2개 발생)

In [61]:
quan_eval_3(delay_data.head(1)['text'],delay_data.head(1)['output'])

(False,
 '\n',
 '질문1: 해당 지하철 지연 사건이 일어난 날짜는 언제인가요?\n(1) 2022-07-15 (2) 2023-01-02 (3) 2021-12-25 (4) 2023-03-20 (5) 알 수 없음\n\n질문2: 이번 지하철 지연 사건이 발생한 호선은 무엇인가요?\n(1) 2호선 (2) 3호선 (3) 1호선 (4) 4호선 (5) 알 수 없음\n\n질문3: 이번 지하철 지연 사건의 주된 원인은 무엇인가요?\n(1) 기계 고장 (2) 노조 준법투쟁 (3) 자연재해 (4) 전력 문제 (5) 알 수 없음',
 '\n',
 '답변1: 5번,\n답변2: 3번,\n답변3: 2번',
 '\n',
 '답변1: 5번,\n답변2: 3번,\n답변3: 5번')

In [64]:
quan_eval_3(delay_data.tail(1)['text'],delay_data.tail(1)['output'])

(False,
 '\n',
 '질문1: 해당 지하철 사건이 일어난 일시는 언제인가요? (1) 2023-01-15 (2) 2023-02-20 (3) 2023-03-25 (4) 2023-04-10 (5) 알 수 없음\n질문2: 해당 지하철 사건이 발생한 호선은 어디인가요? (1) 1호선 (2) 2호선 (3) 4호선 (4) 7호선 (5) 알 수 없음\n질문3: 해당 지하철 사건의 발생 원인은 무엇인가요? (1) 전기 고장 (2) 신호 장애 (3) 열차 고장 (4) 승객 쓰러짐 (5) 알 수 없음',
 '\n',
 '답변1: 5번,\n답변2: 3번,\n답변3: 4번',
 '\n',
 '답변1: 5번,\n답변2: 3번,\n답변3: 5번')

### test #3 시간표글에 적용(📌False 1개 발생)

In [62]:
quan_eval_3(time_data.head(1)['text'],time_data.head(1)['output'])

(True,
 '\n',
 '질문1: 해당 지하철 사건이 일어난 날짜는 언제입니까?\n(1) 2024-01-15 (2) 2024-01-20 (3) 2024-01-25 (4) 2024-02-01 (5) 알 수 없음\n\n질문2: 사건이 발생한 지하철 호선은 무엇입니까?\n(1) 1호선 (2) 2호선 (3) 3호선 (4) 4호선 (5) 알 수 없음\n\n질문3: 사건 발생의 주된 원인은 무엇입니까?\n(1) 신호 장애 (2) 열차 고장 (3) 승객 사고 (4) 기상 악화 (5) 알 수 없음',
 '\n',
 '답변1: 5번,\n답변2: 1번,\n답변3: 5번',
 '\n',
 '답변1: 5번,\n답변2: 1번,\n답변3: 5번')

In [65]:
quan_eval_3(time_data.tail(1)['text'],time_data.tail(1)['output'])

(False,
 '\n',
 '질문1: 해당 지하철 사건이 일어난 날짜는 언제인가요? (1) 2024-04-29 (2) 2024-04-30 (3) 2024-05-01 (4) 2024-05-02 (5) 알 수 없음\n\n질문2: 사건이 발생한 지하철 호선은 무엇인가요? (1) 2호선 (2) 3호선 (3) 5호선 (4) 7호선 (5) 알 수 없음\n\n질문3: 사건 발생 원인은 무엇인가요? (1) 전기 고장 (2) 신호 장애 (3) 운행 시각표 변경 (4) 철로 이상 (5) 알 수 없음',
 '\n',
 '답변1: 2번,\n답변2: 4번,\n답변3: 3번',
 '\n',
 '답변1: 5번,\n답변2: 4번,\n답변3: 5번')

## **6-4. 결론2 + 발견한 문제**
1. **5번(알 수 없음)을 답으로 고르는 경우 다수**
2. **질문 3번: '사건 발생 원인'에서 틀리는 경우 다수**


## **6-5. 수정본2 단일 셀(폐기..)**
1. **선지의 개수를 3개로 감소**
2. **선지에 (3) '알 수 없음' 무조건 포함은 - 유지**
3. **일자정보 질문의 선지 형식 2018-01-01 형식으로 통일 - 유지**
4. [기타] (사건 일시, 사건 지하철 호선, 사건 발생 원인) -> (.... 사건 발생 원인 ***등***) 으로 변경


In [75]:
# PROMPT1 - 뉴스 기반으로 질문 생성 - chain_1으로 생성
prompt_1 = ChatPromptTemplate.from_messages([
    ("system", """너는 뉴스 기사를 읽고, 뉴스의 핵심 내용들에 대해서 3지선다 질문을 던지는 것을 잘 해."""), # 페르소나 부여
    ("user", """제시된 서울 지하철 뉴스를 읽고, 뉴스의 주요 내용(사건 일시, 사건 지하철 호선, 사건 발생 원인 등)을 확인하기 위한 3가지 질문을 만들어줘. 이 때, 질문은 1번부터 3번까지의 선지 중에서 하나를 고르는 3지선다형 문제여야만해.
    예시= '해당 지하철 사건이 일어난 발생지는 어디인가요? (1)강남역 (2)성수역 (3)알 수 없음). 단, 세개의 선지 중 하나는 반드시 '알 수 없음' 이어야 하고, '사건 일시' 질문의 선지는 2018-03-08 형식을 갖춰야해.
    뉴스= {input}
    """),
    ("system", """템플릿:
    질문1: {{question_1}},
    질문2: {{question_2}},
    질문3: {{question_3}}""")
])
chain_1 = prompt_1 | llm | output_parser

# PROMPT2 입력 - 뉴스 기사에 대해 질문의 답변 도출 - chain_2으로 생성
prompt_2 = ChatPromptTemplate.from_messages([
    ("system", """너는 뉴스 기사를 읽고, 읽은 내용을 바탕으로 질문에 대해 정확히 답변해"""), # 페르소나 부여
    ("user", """제시된 서울 지하철 뉴스와 해당 뉴스에 대한 3가지 질문에 대해 답변해줘.
    예시= "1번, 1번, 2번",
    뉴스= {input}
    질문= {question}
    """),
    ("system", """템플릿:
    답변1: {{answer_1}}번,
    답변2: {{answer_2}}번,
    답변3: {{answer_3}}번""")
])
chain_2 = prompt_2 | llm | output_parser

# PROMPT3 입력 - 블로그글에 대해 질문의 답변 도출 - chain_3으로 생성
prompt_3 = ChatPromptTemplate.from_messages([
    ("system", """너는 블로그 글을 읽고, 읽은 내용을 바탕으로 질문에 대해 정확히 답변해"""), # 페르소나 부여
    ("user", """제시된 서울 지하철 뉴스와 해당 뉴스에 대한 3가지 질문에 대해 답변해줘.
    예시= "1번, 3번, 2번",
    블로그 글= {input}
    질문= {question}
    """),
    ("system", """템플릿:
    답변1: {{answer_1}}번,
    답변2: {{answer_2}}번,
    답변3: {{answer_3}}번""")
])
chain_3 = prompt_3 | llm | output_parser


# 결과까지 다 볼 수 있게 함수 새로 정의
def quan_eval_3(news, blog):

  # news 기반으로 질문 3개 생성
  question_list = chain_1.invoke({"input": news})

  # 질문 3개에 대해 news 기반으로 답변
  answer_list_news = chain_2.invoke({"input": news, "question": question_list})

  # 질문 3개에 대해 blog 기반으로 답변
  answer_list_blog = chain_3.invoke({"input": blog, "question": question_list})

  # 일치 여부 판단(True/False)
  return answer_list_news == answer_list_blog, '\n', question_list, '\n', answer_list_news,'\n', answer_list_blog





### test #1. 파업글에 적용

In [77]:
quan_eval_3(strike_data.head(1)['text'],strike_data.head(1)['output'])

(True,
 '\n',
 '질문1: 해당 지하철 사건이 일어난 날짜는 언제입니까? (1) 2022-05-15 (2) 2023-01-20 (3) 알 수 없음\n질문2: 이 사건이 발생한 지하철 호선은 어디입니까? (1) 2호선 (2) 9호선 (3) 알 수 없음\n질문3: 이 지하철 사건의 주요 원인은 무엇입니까? (1) 전기 공급 문제 (2) 파업 (3) 알 수 없음',
 '\n',
 '답변1: 3번,\n답변2: 3번,\n답변3: 2번',
 '\n',
 '답변1: 3번,\n답변2: 3번,\n답변3: 2번')

In [78]:
quan_eval_3(strike_data.head(1)['text'],strike_data.head(1)['output'])

(True,
 '\n',
 '질문1: 해당 지하철 파업 사건이 일어난 날짜는 언제입니까? (1) 2023-09-25 (2) 2023-10-05 (3) 알 수 없음\n질문2: 이번 지하철 파업이 발생한 호선은 어디입니까? (1) 2호선 (2) 3호선 (3) 알 수 없음\n질문3: 이번 지하철 문제의 주된 원인은 무엇인가요? (1) 기술적 결함 (2) 파업 (3) 알 수 없음',
 '\n',
 '답변1: 3번,\n답변2: 3번,\n답변3: 2번',
 '\n',
 '답변1: 3번,\n답변2: 3번,\n답변3: 2번')

### test #2. 지연글에 적용

In [79]:
quan_eval_3(delay_data.head(1)['text'],delay_data.head(1)['output'])

(False,
 '\n',
 '템플릿:\n    질문1: 해당 지하철 사건이 일어난 요일은 무엇인가요? (1)화요일 (2)월요일 (3)알 수 없음\n    질문2: 사건이 발생한 지하철 호선은 무엇인가요? (1)2호선 (2)1호선 (3)알 수 없음\n    질문3: 지하철 지연의 원인은 무엇이었나요? (1)노조 파업 (2)노조 준법투쟁 (3)알 수 없음',
 '\n',
 '답변1: 2번,\n답변2: 2번,\n답변3: 2번',
 '\n',
 '답변1: 3번,\n답변2: 2번,\n답변3: 3번')

In [80]:
quan_eval_3(delay_data.head(1)['text'],delay_data.head(1)['output'])

(False,
 '\n',
 '질문1: 지하철 지연 운행이 발생한 요일은 언제인가요? \n(1) 월요일 (2) 금요일 (3) 알 수 없음\n\n질문2: 지연 운행이 발생한 서울 지하철의 호선은 무엇인가요?\n(1) 1호선 (2) 2호선 (3) 알 수 없음\n\n질문3: 이번 지하철 지연 운행의 원인은 무엇인가요?\n(1) 노조 준법투쟁 (2) 기계적 고장 (3) 알 수 없음',
 '\n',
 '답변1: 1번,\n답변2: 1번,\n답변3: 1번',
 '\n',
 '답변1: 3번,\n답변2: 1번,\n답변3: 3번')

### test #3. 시간표글에 적용

In [83]:
quan_eval_3(time_data.tail(1)['text'],time_data.tail(1)['output'])

(False,
 '\n',
 '질문1: 이 안내문이 작성된 날짜는 언제인가요? (1) 2024-04-29 (2) 2024-04-30 (3) 알 수 없음\n질문2: 안내문의 내용은 어떤 지하철 호선과 관련이 있나요? (1) 7호선 (2) 9호선 (3) 알 수 없음\n질문3: 안내문의 주요 내용은 무엇인가요? (1) 열차운행 시각표 변경 (2) 역사 리모델링 공지 (3) 알 수 없음',
 '\n',
 '답변1: 2번,\n답변2: 1번,\n답변3: 1번',
 '\n',
 '답변1: 3번,\n답변2: 1번,\n답변3: 1번')

In [84]:
quan_eval_3(time_data.tail(1)['text'],time_data.tail(1)['output'])

(False,
 '\n',
 '질문1: 해당 지하철 사건의 발생 일시는 언제인가요? (1) 2024-04-30 (2) 2023-12-25 (3) 알 수 없음\n질문2: 사건이 발생한 지하철 호선은 무엇인가요? (1) 2호선 (2) 7호선 (3) 알 수 없음\n질문3: 사건의 주요 원인은 무엇인가요? (1) 운행 시각표 변경 (2) 기계 고장 (3) 알 수 없음',
 '\n',
 '답변1: 1번,\n답변2: 2번,\n답변3: 1번',
 '\n',
 '답변1: 3번,\n답변2: 2번,\n답변3: 3번')

### **✦ 논의사항**
- 100% 정확하게 만드는 것은 사실상 불가능
- 차라리 총점에서 감점시키고 임계값 넘으면 통과, 아니면 기각 - 이런식으로 평가하는 건 어떤지?